In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transformers
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm
import sklearn
from sklearn.preprocessing import normalize
from tqdm.notebook import tqdm

import glob
import time


import os

#import tensorflow as tf

In [2]:
initial_states = []

for dir, sub_dir, files in os.walk("input_data/"):
    for file in sorted(files):
        print(file)
        temp = pd.read_csv(("input_data/"+file),index_col=None, header=0)
        initial_states.append(temp)
(initial_states)

00000_to_02284-initial_states.csv
02285_to_02357-initial_states.csv
02358_to_04264-initial_states.csv
04265_to_05570-initial_states.csv
05571_to_05614-initial_states.csv
05615_to_06671-initial_states.csv
06672_to_08118-initial_states.csv


[      File ID            Timestamp  Semi-major Axis (km)  Eccentricity  \
 0           0  2000-08-02 04:50:33           6826.387247      0.003882   
 1           1  2000-08-03 19:51:01           6826.327748      0.003879   
 2           2  2000-08-05 05:40:05           6819.634802      0.004114   
 3           3  2000-08-06 05:02:20           6819.606603      0.004134   
 4           4  2000-08-08 20:54:57           6819.425918      0.004178   
 ...       ...                  ...                   ...           ...   
 2280     2280  2010-07-24 12:36:39           6643.025072      0.000418   
 2281     2281  2010-07-26 12:32:28           6642.391773      0.000418   
 2282     2282  2010-07-27 00:31:21           6642.225069      0.000418   
 2283     2283  2010-07-29 12:25:24           6641.354564      0.000439   
 2284     2284  2010-07-30 04:53:37           6641.071888      0.000419   
 
       Inclination (deg)  RAAN (deg)  Argument of Perigee (deg)  \
 0             87.275306  144.1

In [3]:
initial_states_df = pd.concat(initial_states,axis=0,ignore_index=True)
initial_states_df

,File ID,Timestamp,Semi-major Axis (km),Eccentricity,Inclination (deg),RAAN (deg),Argument of Perigee (deg),True Anomaly (deg),Latitude (deg),Longitude (deg),Altitude (km)
0,0,2000-08-02 04:50:33,6826.387247,0.003882,87.275306,144.135111,257.314389,102.383270,43.637815,-62.543128,466.448890
1,1,2000-08-03 19:51:01,6826.327748,0.003879,87.275694,143.529694,250.438806,109.273118,43.444458,70.139709,463.435053
2,2,2000-08-05 05:40:05,6819.634802,0.004114,87.268611,142.972111,244.549389,115.138737,19.764250,104.521278,471.625453
3,3,2000-08-06 05:02:20,6819.606603,0.004134,87.268194,142.608389,241.172000,118.545161,12.450738,112.239558,470.385914
4,4,2000-08-08 20:54:57,6819.425918,0.004178,87.264611,141.605111,228.779611,130.982981,-8.776992,-130.559634,468.911226
...,...,...,...,...,...,...,...,...,...,...,...
8114,8114,2019-12-25 00:00:00,6765.013678,0.005730,87.863978,102.587920,240.608198,187.758342,69.535173,108.291937,443.930606
8115,8115,2019-12-27 00:00:00,6801.130577,0.002172,90.690901,99.760357,152.602156,226.350702,20.225336,99.507926,435.492910
8116,8116,2019-12-28 00:00:00,6805.864837,0.001925,91.053632,96.918243,43.442569,131.160787,4.265831,-83.003782,436.652863
8117,8117,2019-12-30 00:00:00,6774.300973,0.004785,88.598951,92.814340,320.652681,164.621884,53.716936,-89.084995,441.438958


In [4]:
hold = initial_states_df
x = initial_states_df.iloc[:,2:].values
x = normalize(x,norm='l2')
hold = pd.concat([hold['File ID'],pd.DataFrame(x)],axis=1)
initial_states_normalized = hold
initial_states_normalized

,File ID,0,1,2,3,4,5,6,7,8
0,0,0.996494,5.666527e-07,0.012740,0.021040,0.037562,0.014946,0.006370,-0.009130,0.068091
1,1,0.996537,5.663169e-07,0.012741,0.020953,0.036560,0.015952,0.006342,0.010239,0.067654
2,2,0.996419,6.010249e-07,0.012751,0.020890,0.035731,0.016823,0.002888,0.015272,0.068909
3,3,0.996426,6.039686e-07,0.012751,0.020837,0.035238,0.017321,0.001819,0.016400,0.068729
4,4,0.996426,6.104722e-07,0.012751,0.020691,0.033428,0.019139,-0.001282,-0.019077,0.068515
...,...,...,...,...,...,...,...,...,...,...
8114,8114,0.996468,8.440400e-07,0.012942,0.015111,0.035441,0.027656,0.010242,0.015951,0.065390
8115,8115,0.996851,3.183163e-07,0.013293,0.014622,0.022367,0.033177,0.002964,0.014585,0.063831
8116,8116,0.997480,2.821553e-07,0.013345,0.014205,0.006367,0.019223,0.000625,-0.012165,0.063997
8117,8117,0.996186,7.035914e-07,0.013029,0.013649,0.047153,0.024208,0.007899,-0.013100,0.064915


In [103]:
initial_states_normalized['File ID'].dtype

dtype('int64')

In [ ]:
#initial_states_normalized = ((initial_states_df.iloc[:,2:] - initial_states_df.iloc[:,2:].mean())/initial_states_df.iloc[:,2:].std())
#initial_states_normalized

In [69]:
dataloader = DataLoader(initial_states_normalized, batch_size=32, shuffle=True)

In [ ]:
import glob

class StaticToFixedLengthTargetDataset(Dataset):
    def __init__(self, initial_states_df, targets_dir="data/dataset/test/sat_density", desired_len=432):
        self.data = initial_states_df.reset_index(drop=True)
        self.targets_dir = targets_dir
        self.desired_len = desired_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        static_input = torch.tensor(row.drop('File ID').values, dtype=torch.float32)
        file_id = str(row['File ID']).zfill(5)  # Ensure it's 5 characters with leading zeros if needed

        # Correct filename pattern: 6-char satellite code + '-' + File ID + '-'
        pattern = os.path.join(self.targets_dir, f"??????-{file_id}-*.csv")
        matched_files = glob.glob(pattern)

        if len(matched_files) == 0:
            raise FileNotFoundError(f"No target file found for File ID {file_id}")
        elif len(matched_files) > 1:
            raise RuntimeError(f"Multiple files matched for File ID {file_id}: {matched_files}")

        target_path = matched_files[0]

        # Load and process the target file
        target_df = pd.read_csv(target_path)
        density = target_df.iloc[:, 1].values
        actual_len = len(density)

        padded = torch.zeros(self.desired_len, dtype=torch.float32)
        mask = torch.zeros(self.desired_len, dtype=torch.bool)

        if actual_len >= self.desired_len:
            padded[:] = torch.tensor(density[:self.desired_len], dtype=torch.float32)
            mask[:] = 1
        else:
            padded[:actual_len] = torch.tensor(density, dtype=torch.float32)
            mask[:actual_len] = 1

        return static_input, padded, mask


In [36]:
class InitialStateEmbedder(nn.Module):
    def __init__(self, input_dim=9, embedding_dim=128):
        super(InitialStateEmbedder, self).__init__()
        self.embed = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.1),
            nn.Linear(64, embedding_dim),
            nn.ReLU()
        )

    def forward(self, x):
        # x shape: (batch_size, 9)
        return self.embed(x)  # output: (batch_size, embedding_dim)

In [70]:
model = InitialStateEmbedder()

In [71]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epoch = 5 #iterations over all data
train_losses, val_losses = [],[]

model = testModel()
model.to(device)

for epoch in range(num_epoch):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc='Train loop'):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)


device(type='cuda', index=0)

In [91]:
# Assumes initial_states_normalized has 8119 rows
num_samples = initial_states_normalized.shape[0]  # should be 8119
output_dim = 3  # Predicting 3-day densities

np.random.seed(1)
# Simulate some dummy values (can be random or zeros)
target_values = pd.DataFrame(np.random.randn(num_samples, output_dim), columns=['day_1', 'day_2', 'day_3'])

# --- Step 2: Convert to Tensors ---
X = torch.tensor(initial_states_normalized.values, dtype=torch.float32)  # shape: (8119, 9)
y = torch.tensor(target_values.values, dtype=torch.float32)              # shape: (8119, 3)

# --- Step 3: Create Dataset and Dataloader ---
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# --- Step 4: Define Model ---
class InitialStateEmbedder(nn.Module):
    def __init__(self, input_dim=9, embedding_dim=128):
        super().__init__()
        self.embed = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.1),
            nn.Linear(64, embedding_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.embed(x)

input_dim = X.shape[1]
embedding_dim = 128

embedder = InitialStateEmbedder(input_dim=input_dim, embedding_dim=embedding_dim).to(device)
regressor = nn.Linear(embedding_dim, output_dim).to(device)
model = nn.Sequential(embedder, regressor).to(device)


# --- Step 5: Training Setup ---
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# --- Step 6: Training Loop ---
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for xb, yb in (dataloader):
        xb = xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {epoch_loss / len(dataloader):.4f}")

Epoch 1: Loss = 1.0080
Epoch 2: Loss = 1.0042
Epoch 3: Loss = 1.0031
Epoch 4: Loss = 1.0021
Epoch 5: Loss = 1.0012
Epoch 6: Loss = 1.0015
Epoch 7: Loss = 1.0012
Epoch 8: Loss = 1.0010
Epoch 9: Loss = 1.0011
Epoch 10: Loss = 1.0005
Epoch 11: Loss = 1.0008
Epoch 12: Loss = 1.0010


KeyboardInterrupt: 

In [39]:


# ============================
# Dataset
# ============================
class StaticToFixedLengthTargetDataset(Dataset):
    def __init__(self, initial_states_df, targets_dir="data/dataset/test/sat_density", desired_len=432):
        self.data = initial_states_df.reset_index(drop=True)
        self.targets_dir = targets_dir
        self.desired_len = desired_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        static_input = torch.tensor(row.drop('File ID').values, dtype=torch.float32)
        file_id = str(int(row['File ID'])).zfill(5)  # Ensure it's 5 digits like "08081"

        # Pattern: <6-char satellite code>-<File ID>-*.csv
        pattern = os.path.join(self.targets_dir, f"??????-{file_id}-*.csv")
        matched_files = glob.glob(pattern)
        #print (matched_files)

        if len(matched_files) == 0:
            raise FileNotFoundError(f"No target file found for File ID {file_id}")
        elif len(matched_files) > 1:
            raise RuntimeError(f"Multiple files matched for File ID {file_id}: {matched_files}")

        target_path = matched_files[0]

        # Load density data from 2nd column
        target_df = pd.read_csv(target_path)
        density = pd.to_numeric(target_df.iloc[:, 1], errors='coerce')
        density = density.clip(lower=0, upper=1e3).fillna(0.0).values
        actual_len = len(density)

        # Pad or truncate to fixed length
        padded = torch.zeros(self.desired_len, dtype=torch.float32)
        mask = torch.zeros(self.desired_len, dtype=torch.bool)
        if actual_len >= self.desired_len:
            padded[:] = torch.tensor(density[:self.desired_len], dtype=torch.float32)
            mask[:] = 1
        else:
            padded[:actual_len] = torch.tensor(density, dtype=torch.float32)
            mask[:actual_len] = 1

        if torch.isnan(padded).any():
            print (f"NaN detected in padded target for File ID {file_id}")

        return static_input, padded, mask

# ============================
# Model
# ============================
class StaticToSequenceModel(nn.Module):
    def __init__(self, input_dim=9, embedding_dim=128, output_len=432):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.1),
            nn.Linear(64, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, output_len)
        )

    def forward(self, x):
        return self.model(x)  # (batch_size, 432)

# ============================
# Masked MSE Loss
# ============================
def masked_mse_loss(preds, targets, mask, eps=1e-8):
    preds = torch.nan_to_num(preds, nan=0.0, posinf=1e3, neginf=0.0)
    targets = torch.nan_to_num(targets, nan=0.0, posinf=1e3, neginf=0.0)

    loss = (preds - targets) ** 2
    loss = loss * mask.float()

    denom = mask.sum().clamp(min=1.0)  # avoid division by zero
    return loss.sum() / denom

# ============================
# Training Loop
# ============================
def train_model(initial_states_df, targets_dir="data/dataset/test/sat_density", num_epochs=5, batch_size=64, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset and Dataloader
    dataset = StaticToFixedLengthTargetDataset(initial_states_df, targets_dir)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Model, optimizer, loss
    model = StaticToSequenceModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        start = time.time()
        model.train()
        epoch_loss = 0.0
        for static_input, target_seq, mask in dataloader:
            static_input = static_input.to(device)
            target_seq = target_seq.to(device)
            mask = mask.to(device)
            if torch.isnan(static_input).any():
                print("NaN detected in static_input")
            
            

            preds = model(static_input)
            preds = torch.nan_to_num(preds, nan=0.0, posinf=1e3, neginf=0.0)
            loss = masked_mse_loss(preds, target_seq, mask)
            print (loss.item())

            if torch.isnan(preds).any() or torch.isnan(target_seq).any():
                print("⚠️ NaNs found in predictions or targets")
            if torch.isinf(preds).any() or torch.isinf(target_seq).any():
                print("⚠️ Infs found in predictions or targets")

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        end = time.time()
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f} - Time per epoch: {(end - start):.4f}")

train_model(initial_states_normalized, lr=0.01)


28021.69140625
14363.4453125
14090.5
36620.46484375
5777.41455078125
19455.025390625
21918.533203125
17455.666015625
9195.009765625
11026.693359375
10261.505859375
13063.8623046875
24919.052734375
21758.896484375
26406.052734375
28172.142578125
20175.396484375
12272.025390625
26948.126953125
17732.1484375
9222.9208984375
21868.943359375
14417.9111328125
20033.720703125
9644.40234375
21626.513671875
25434.49609375
12749.986328125
17761.0234375
14656.7451171875
12758.6923828125
10766.94921875
27642.2890625
13528.1611328125
19931.908203125
22102.392578125
11143.6962890625
11000.3994140625
24376.427734375
28503.533203125
28824.07421875
11837.4580078125
14904.0009765625
15849.8310546875
24148.212890625
21173.578125
12422.90234375
24034.279296875
31939.107421875
18433.171875
27821.67578125
14767.837890625
25433.240234375
24150.580078125
13648.037109375
23455.94140625
6841.58642578125
13320.1806640625
15932.8212890625
6432.7685546875
12198.787109375
14764.3271484375
16360.794921875
18337.2011

In [429]:
class FullDataset(Dataset):
    def __init__(self, initial_states_df, density_dir="data/dataset/test/sat_density", goes_dir="data/dataset/test/goes",
                 omni2_dir="data/dataset/test/omni2"):
        self.data = initial_states_df.reset_index(drop=True)
        self.density_dir = density_dir
        self.goes_dir = goes_dir
        self.omni2_dir = omni2_dir
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        static_input = torch.tensor(row.drop('File ID').fillna(0.0).values, dtype=torch.float32)
        
        file_id = str(int(row['File ID'])).zfill(5)
        
        density_file = glob.glob(os.path.join(self.density_dir, f"??????-{file_id}-*.csv"))
        goes_file = glob.glob(os.path.join(self.goes_dir, f"*-{file_id}-*.csv"))
        omni2_file = glob.glob(os.path.join(self.omni2_dir, f"*-{file_id}-*.csv"))

        density_sat = density_file[0].strip(self.density_dir)[0:6]
    
        density_df = ((pd.read_csv(density_file[0])))
        if density_df.shape[0] > 432:
            print("change 1")
            density_df = density_df[:432]
            print(density_df)
        elif density_df.shape[0] < 432:
            print("change 2")
            padding = pd.DataFrame(np.empty((432-density_df.shape[0],2)),columns=density_df.columns)
            padding[:] = np.nan
            density_df = pd.concat((density_df,padding),ignore_index=True)
        density_df_mask = (pd.notnull(density_df)).astype(int)
        density_tensor = torch.tensor(density_df['Orbit Mean Density (kg/m^3)'].fillna(0.0).values)
        density_df_mask_tensor = torch.tensor(density_df_mask.iloc[:,1].values)
        density_stacked = torch.stack((density_tensor,density_df_mask_tensor))
        print("ID", idx)
        print(density_df_mask_tensor)
        print(density_stacked.shape)


        goes_df = pd.read_csv(goes_file[0])
        if goes_df.shape[0] > 86400:
            goes_df = goes_df[goes_df.shape[0]-86400:goes_df.shape[0]]
        elif goes_df.shape[0] < 86400:
            padding = pd.DataFrame(np.empty((86400-goes_df.shape[0],43)),columns=goes_df.columns)
            padding[:] = np.nan
            goes_df = pd.concat((padding,goes_df),ignore_index=True)
        goes_mask = (~pd.isnull(goes_df)).astype(int)
        goes_valid_mask = (goes_df['xrsa_flag'] == 0.0) & (goes_df['xrsb_flag'] == 0.0).astype(int)
        goes_mask = goes_mask.mul(goes_valid_mask.values,axis=0)
        goes_tensor = torch.tensor(normalize(goes_df.iloc[:, 1:].fillna(0.0).values, norm='l2'))
        goes_mask_tensor = torch.tensor(goes_mask.iloc[:, 1:].values)
        goes_stacked = torch.stack((goes_tensor,goes_mask_tensor))
        
        omni2_df = pd.read_csv(omni2_file[0])
        if omni2_df.shape[0] > 1440:
            omni2_df = omni2_df[omni2_df.shape[0]-1440:omni2_df.shape[0]]
        elif goes_df.shape[0] < 1440:
            padding = pd.DataFrame(np.empty((1440-omni2_df.shape[0],58)),columns=omni2_df.columns)
            padding[:] = np.nan
            goes_df = pd.concat((padding,goes_df),ignore_index=True)
        omni2_tensor = torch.tensor(normalize(omni2_df.iloc[:, :57].fillna(0.0).values.astype(float), norm='l2'))
        omni2_mask = (~pd.isnull(omni2_df)).astype(int)
        omni2_mask_tensor = torch.tensor(omni2_mask.iloc[:, :57].values)
        omni2_stacked = torch.stack((omni2_tensor,omni2_mask_tensor))


        return static_input, density_tensor, goes_tensor, omni2_tensor, density_sat



In [430]:
dataset = FullDataset(initial_states_normalized)
dataloader = DataLoader(dataset,batch_size=32,shuffle=False)

#dataset[1]

for static_input, density_tensor, goes_tensor, omni2_tensor, density_sat in dataloader:
    print(density_sat[0])
    
    

ID 0
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 

KeyboardInterrupt: 